<a href="https://colab.research.google.com/github/newmantldr/MUTS25/blob/main/MUTS25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import gspread

In [24]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [25]:
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

## **Data Loads**

In [26]:
## movie info load

url = 'https://docs.google.com/spreadsheets/d/16OdmqJleXa81GahdElnu6dsZg6dY-pP8Bbvd60vgIiY/export?format=csv'
movieINFO = pd.read_csv(url)

In [27]:
## food and bev sales load

url = 'https://docs.google.com/spreadsheets/d/16OdmqJleXa81GahdElnu6dsZg6dY-pP8Bbvd60vgIiY/export?format=csv&gid=1513429195'
foodNBev = pd.read_csv(url)


In [28]:
## placer load

url = 'https://docs.google.com/spreadsheets/d/16OdmqJleXa81GahdElnu6dsZg6dY-pP8Bbvd60vgIiY/export?format=csv&gid=1092853506'
placer = pd.read_csv(url)

In [29]:
## marketing

url = 'https://docs.google.com/spreadsheets/d/16OdmqJleXa81GahdElnu6dsZg6dY-pP8Bbvd60vgIiY/export?format=csv&gid=61019128'
marketing = pd.read_csv(url)

In [30]:
## ticket data

url = 'https://docs.google.com/spreadsheets/d/16OdmqJleXa81GahdElnu6dsZg6dY-pP8Bbvd60vgIiY/export?format=csv&gid=1883735805'
rawTXT = pd.read_csv(url)

Ideas:

find comparisons of movie types, audiences and sales <br>
find zip breakdowns (apt numbers?) <br>
find estimated ethnicity breakdown <br>
size of ticket groups <br>
times tickets are sold most <br>
most successful marketing conversions <br>
who is the normal consumer? <br>
by movie breakdowns <br>
correlations between movie and turnout and sales? <br>






In [31]:
## Isolate MUTS sales, remove concession sales, and keep only lines where we
# have buyer information

moviesTXT = rawTXT[rawTXT['event.name'] == 'Movies Under The Stars']
moviesTXT = moviesTXT[moviesTXT['category'] != 'Concessions']
moviesTXT = moviesTXT.dropna(subset = ['customer.postal'])
moviesTXT

,name,firstname,lastname,event.name,event.start,customerId,transactionId,email,category,price,origin,createdAt,updatedAt,seat.section,seat.seat,seat.general_admission,customer.street,customer.city,customer.postal,customer.country,customer.state,checkinTimes,checkin
0,Sam Van Otterloo,Sam,Van Otterloo,Movies Under The Stars,2025-10-18T02:00:16.111Z,68bef628483341d126182e38,68bef642c0a05e5f41213d8e,ais5201@gmail.com,General Seating,10.0,online,2025-09-08T15:27:13.844Z,2025-09-08T15:31:04.886Z,NaN,NaN,GA Seating,1503 DANTE CIRCLE,ROSEVILLE,95678,US,CA,NaN,False
1,Anna Schneider,Anna,Schneider,Movies Under The Stars,2025-10-18T02:00:16.111Z,68bef628483341d126182e38,68bef642c0a05e5f41213d8e,ais5201@gmail.com,General Seating,10.0,online,2025-09-08T15:27:13.844Z,2025-09-08T15:30:23.541Z,NaN,NaN,GA Seating,1503 DANTE CIRCLE,ROSEVILLE,95678,US,CA,2025-10-18T01:35:55.914Z - DSP Event Scanner,True
2,Kathryn Mueller,Kathryn,Mueller,Movies Under The Stars,2025-10-18T02:00:16.111Z,68bef628483341d126182e38,68bef642c0a05e5f41213d8e,ais5201@gmail.com,General Seating,10.0,online,2025-09-08T15:27:13.844Z,2025-09-08T15:29:46.435Z,NaN,NaN,GA Seating,1503 DANTE CIRCLE,ROSEVILLE,95678,US,CA,NaN,False
3,Anna Schneider,Anna,Schneider,Movies Under The Stars,2025-10-25T02:00:16.111Z,68bef628483341d126182e38,68bef74e4204ce90a26bc976,ais5201@gmail.com,General Seating,10.0,online,2025-09-08T15:32:37.171Z,2025-09-08T15:33:49.617Z,NaN,NaN,GA Seating,1503 DANTE CIRCLE,ROSEVILLE,95678,US,CA,NaN,False
4,Sam Van Otterloo,Sam,Van Otterloo,Movies Under The Stars,2025-10-25T02:00:16.111Z,68bef628483341d126182e38,68bef74e4204ce90a26bc976,ais5201@gmail.com,General Seating,10.0,online,2025-09-08T15:32:37.171Z,2025-09-08T15:34:19.075Z,NaN,NaN,GA Seating,1503 DANTE CIRCLE,ROSEVILLE,95678,US,CA,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2791,Richard V,Richard,V,Movies Under The Stars,2025-11-01T02:00:16.111Z,69054514aa3117d6372f4e7d,6905452d90196d5eed8a6774,salty.fumble5q@icloud.com,Preferred Seating,25.0,online,2025-10-31T23:22:44.694Z,2025-10-31T23:24:29.947Z,NaN,NaN,Preferred Seating,8673 Duryea Drive,Sacramento,95828,US,CA,2025-11-01T02:28:55.556Z - DSP Event Scanner,True
2792,Melody Chan,Melody,Chan,Movies Under The Stars,2025-11-01T02:00:16.111Z,6905510740f2e9904ff31302,6905513a7d29a75e26381cf3,melchan5059@gmail.com,Preferred Seating,25.0,online,2025-11-01T00:12:22.282Z,2025-11-01T00:15:54.749Z,NaN,NaN,Preferred Seating,1560 Zelda Way,Sacramento,95822,US,CA,2025-11-01T01:21:45.583Z - DSP Event Scanner,True
2793,Julian Gonzales,Julian,Gonzales,Movies Under The Stars,2025-11-01T02:00:16.111Z,690553ff025f98e6f29ed816,6905540eec78f75ee59ef615,gonzales.ju@icloud.com,Preferred Seating,25.0,online,2025-11-01T00:25:24.939Z,2025-11-01T00:27:58.289Z,NaN,NaN,Preferred Seating,2111 Arliss Way,Sacramento,95822,US,CA,2025-11-01T01:01:07.888Z - DSP Event Scanner |...,True
2794,Julian Gonzales,Julian,Gonzales,Movies Under The Stars,2025-11-01T02:00:16.111Z,690553ff025f98e6f29ed816,6905540eec78f75ee59ef615,gonzales.ju@icloud.com,Preferred Seating,25.0,online,2025-11-01T00:25:24.940Z,2025-11-01T00:27:58.289Z,NaN,NaN,Preferred Seating,2111 Arliss Way,Sacramento,95822,US,CA,2025-11-01T01:01:07.890Z - DSP Event Scanner,True


In [32]:
## Localize and split time into dt

moviesTXT['createdAt'] = pd.to_datetime(moviesTXT['createdAt'])
moviesTXT['createdAt'] = moviesTXT['createdAt'].dt.tz_convert('America/Los_Angeles').dt.tz_localize(None)
moviesTXT['createdDate'] = moviesTXT['createdAt'].dt.date
moviesTXT['createdTime'] = moviesTXT['createdAt'].dt.time
moviesTXT['createdDOTW'] = moviesTXT['createdAt'].dt.day_name()
moviesTXT

,name,firstname,lastname,event.name,event.start,customerId,transactionId,email,category,price,origin,createdAt,updatedAt,seat.section,seat.seat,seat.general_admission,customer.street,customer.city,customer.postal,customer.country,customer.state,checkinTimes,checkin,createdDate,createdTime,createdDOTW
0,Sam Van Otterloo,Sam,Van Otterloo,Movies Under The Stars,2025-10-18T02:00:16.111Z,68bef628483341d126182e38,68bef642c0a05e5f41213d8e,ais5201@gmail.com,General Seating,10.0,online,2025-09-08 08:27:13.844,2025-09-08T15:31:04.886Z,NaN,NaN,GA Seating,1503 DANTE CIRCLE,ROSEVILLE,95678,US,CA,NaN,False,2025-09-08,08:27:13.844000,Monday
1,Anna Schneider,Anna,Schneider,Movies Under The Stars,2025-10-18T02:00:16.111Z,68bef628483341d126182e38,68bef642c0a05e5f41213d8e,ais5201@gmail.com,General Seating,10.0,online,2025-09-08 08:27:13.844,2025-09-08T15:30:23.541Z,NaN,NaN,GA Seating,1503 DANTE CIRCLE,ROSEVILLE,95678,US,CA,2025-10-18T01:35:55.914Z - DSP Event Scanner,True,2025-09-08,08:27:13.844000,Monday
2,Kathryn Mueller,Kathryn,Mueller,Movies Under The Stars,2025-10-18T02:00:16.111Z,68bef628483341d126182e38,68bef642c0a05e5f41213d8e,ais5201@gmail.com,General Seating,10.0,online,2025-09-08 08:27:13.844,2025-09-08T15:29:46.435Z,NaN,NaN,GA Seating,1503 DANTE CIRCLE,ROSEVILLE,95678,US,CA,NaN,False,2025-09-08,08:27:13.844000,Monday
3,Anna Schneider,Anna,Schneider,Movies Under The Stars,2025-10-25T02:00:16.111Z,68bef628483341d126182e38,68bef74e4204ce90a26bc976,ais5201@gmail.com,General Seating,10.0,online,2025-09-08 08:32:37.171,2025-09-08T15:33:49.617Z,NaN,NaN,GA Seating,1503 DANTE CIRCLE,ROSEVILLE,95678,US,CA,NaN,False,2025-09-08,08:32:37.171000,Monday
4,Sam Van Otterloo,Sam,Van Otterloo,Movies Under The Stars,2025-10-25T02:00:16.111Z,68bef628483341d126182e38,68bef74e4204ce90a26bc976,ais5201@gmail.com,General Seating,10.0,online,2025-09-08 08:32:37.171,2025-09-08T15:34:19.075Z,NaN,NaN,GA Seating,1503 DANTE CIRCLE,ROSEVILLE,95678,US,CA,NaN,False,2025-09-08,08:32:37.171000,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2791,Richard V,Richard,V,Movies Under The Stars,2025-11-01T02:00:16.111Z,69054514aa3117d6372f4e7d,6905452d90196d5eed8a6774,salty.fumble5q@icloud.com,Preferred Seating,25.0,online,2025-10-31 16:22:44.694,2025-10-31T23:24:29.947Z,NaN,NaN,Preferred Seating,8673 Duryea Drive,Sacramento,95828,US,CA,2025-11-01T02:28:55.556Z - DSP Event Scanner,True,2025-10-31,16:22:44.694000,Friday
2792,Melody Chan,Melody,Chan,Movies Under The Stars,2025-11-01T02:00:16.111Z,6905510740f2e9904ff31302,6905513a7d29a75e26381cf3,melchan5059@gmail.com,Preferred Seating,25.0,online,2025-10-31 17:12:22.282,2025-11-01T00:15:54.749Z,NaN,NaN,Preferred Seating,1560 Zelda Way,Sacramento,95822,US,CA,2025-11-01T01:21:45.583Z - DSP Event Scanner,True,2025-10-31,17:12:22.282000,Friday
2793,Julian Gonzales,Julian,Gonzales,Movies Under The Stars,2025-11-01T02:00:16.111Z,690553ff025f98e6f29ed816,6905540eec78f75ee59ef615,gonzales.ju@icloud.com,Preferred Seating,25.0,online,2025-10-31 17:25:24.939,2025-11-01T00:27:58.289Z,NaN,NaN,Preferred Seating,2111 Arliss Way,Sacramento,95822,US,CA,2025-11-01T01:01:07.888Z - DSP Event Scanner |...,True,2025-10-31,17:25:24.939000,Friday
2794,Julian Gonzales,Julian,Gonzales,Movies Under The Stars,2025-11-01T02:00:16.111Z,690553ff025f98e6f29ed816,6905540eec78f75ee59ef615,gonzales.ju@icloud.com,Preferred Seating,25.0,online,2025-10-31 17:25:24.940,2025-11-01T00:27:58.289Z,NaN,NaN,Preferred Seating,2111 Arliss Way,Sacramento,95822,US,CA,2025-11-01T01:01:07.890Z - DSP Event Scanner,True,2025-10-31,17:25:24.940000,Friday


In [33]:
pip install ethnicolr

In [34]:
## Predict race with ethnicolr package

from ethnicolr import census_ln, pred_census_ln

moviesTXT = pred_census_ln(moviesTXT, 'lastname')
moviesTXT

,name,firstname,lastname,event.name,event.start,customerId,transactionId,email,category,price,origin,createdAt,updatedAt,seat.section,seat.seat,seat.general_admission,customer.street,customer.city,customer.postal,customer.country,customer.state,checkinTimes,checkin,createdDate,createdTime,createdDOTW,api,black,hispanic,white,race
0,Sam Van Otterloo,Sam,Van Otterloo,Movies Under The Stars,2025-10-18T02:00:16.111Z,68bef628483341d126182e38,68bef642c0a05e5f41213d8e,ais5201@gmail.com,General Seating,10.0,online,2025-09-08 08:27:13.844,2025-09-08T15:31:04.886Z,NaN,NaN,GA Seating,1503 DANTE CIRCLE,ROSEVILLE,95678,US,CA,NaN,False,2025-09-08,08:27:13.844000,Monday,0.003171,0.042621,0.041967,0.912241,white
1,Anna Schneider,Anna,Schneider,Movies Under The Stars,2025-10-18T02:00:16.111Z,68bef628483341d126182e38,68bef642c0a05e5f41213d8e,ais5201@gmail.com,General Seating,10.0,online,2025-09-08 08:27:13.844,2025-09-08T15:30:23.541Z,NaN,NaN,GA Seating,1503 DANTE CIRCLE,ROSEVILLE,95678,US,CA,2025-10-18T01:35:55.914Z - DSP Event Scanner,True,2025-09-08,08:27:13.844000,Monday,0.010956,0.003589,0.020190,0.965265,white
2,Kathryn Mueller,Kathryn,Mueller,Movies Under The Stars,2025-10-18T02:00:16.111Z,68bef628483341d126182e38,68bef642c0a05e5f41213d8e,ais5201@gmail.com,General Seating,10.0,online,2025-09-08 08:27:13.844,2025-09-08T15:29:46.435Z,NaN,NaN,GA Seating,1503 DANTE CIRCLE,ROSEVILLE,95678,US,CA,NaN,False,2025-09-08,08:27:13.844000,Monday,0.004509,0.002017,0.025465,0.968009,white
3,Anna Schneider,Anna,Schneider,Movies Under The Stars,2025-10-25T02:00:16.111Z,68bef628483341d126182e38,68bef74e4204ce90a26bc976,ais5201@gmail.com,General Seating,10.0,online,2025-09-08 08:32:37.171,2025-09-08T15:33:49.617Z,NaN,NaN,GA Seating,1503 DANTE CIRCLE,ROSEVILLE,95678,US,CA,NaN,False,2025-09-08,08:32:37.171000,Monday,0.010956,0.003589,0.020190,0.965265,white
4,Sam Van Otterloo,Sam,Van Otterloo,Movies Under The Stars,2025-10-25T02:00:16.111Z,68bef628483341d126182e38,68bef74e4204ce90a26bc976,ais5201@gmail.com,General Seating,10.0,online,2025-09-08 08:32:37.171,2025-09-08T15:34:19.075Z,NaN,NaN,GA Seating,1503 DANTE CIRCLE,ROSEVILLE,95678,US,CA,NaN,False,2025-09-08,08:32:37.171000,Monday,0.003171,0.042621,0.041967,0.912241,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2722,Richard V,Richard,V,Movies Under The Stars,2025-11-01T02:00:16.111Z,69054514aa3117d6372f4e7d,6905452d90196d5eed8a6774,salty.fumble5q@icloud.com,Preferred Seating,25.0,online,2025-10-31 16:22:44.694,2025-10-31T23:24:29.947Z,NaN,NaN,Preferred Seating,8673 Duryea Drive,Sacramento,95828,US,CA,2025-11-01T02:28:55.556Z - DSP Event Scanner,True,2025-10-31,16:22:44.694000,Friday,0.461375,0.068442,0.085889,0.384294,api
2723,Melody Chan,Melody,Chan,Movies Under The Stars,2025-11-01T02:00:16.111Z,6905510740f2e9904ff31302,6905513a7d29a75e26381cf3,melchan5059@gmail.com,Preferred Seating,25.0,online,2025-10-31 17:12:22.282,2025-11-01T00:15:54.749Z,NaN,NaN,Preferred Seating,1560 Zelda Way,Sacramento,95822,US,CA,2025-11-01T01:21:45.583Z - DSP Event Scanner,True,2025-10-31,17:12:22.282000,Friday,0.934207,0.008811,0.026576,0.030405,api
2724,Julian Gonzales,Julian,Gonzales,Movies Under The Stars,2025-11-01T02:00:16.111Z,690553ff025f98e6f29ed816,6905540eec78f75ee59ef615,gonzales.ju@icloud.com,Preferred Seating,25.0,online,2025-10-31 17:25:24.939,2025-11-01T00:27:58.289Z,NaN,NaN,Preferred Seating,2111 Arliss Way,Sacramento,95822,US,CA,2025-11-01T01:01:07.888Z - DSP Event Scanner |...,True,2025-10-31,17:25:24.939000,Friday,0.029108,0.008559,0.882488,0.079846,hispanic
2725,Julian Gonzales,Julian,Gonzales,Movies Under The Stars,2025-11-01T02:00:16.111Z,690553ff025f98e6f29ed816,6905540eec78f75ee59ef615,gonzales.ju@icloud.com,Preferred Seating,25.0,online,2025-10-31 17:25:24.940,2025-11-01T00:27:58.289Z,NaN,NaN,Preferred Seating,2111 Arliss Way,Sacramento,95822,US,CA,2025-11-01T01:01:07.890Z - DSP Event Scanner,True,2025-10-31,17:25:

In [36]:
## Group by predicted race and find percentage of each race attend

raceGrpd = moviesTXT.groupby('race').count()
raceGrpd['name']/raceGrpd['name'].sum()

,name,firstname,lastname,event.name,event.start,customerId,transactionId,email,category,price,origin,createdAt,updatedAt,seat.section,seat.seat,seat.general_admission,customer.street,customer.city,customer.postal,customer.country,customer.state,checkinTimes,checkin,createdDate,createdTime,createdDOTW,api,black,hispanic,white
race,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
api,192,192,192,192,192,192,192,192,192,192,192,192,192,35,35,157,192,192,192,192,192,144,192,192,192,192,192,192,192,192
black,50,50,50,50,50,50,50,50,50,50,50,50,50,4,4,46,50,50,50,50,50,33,50,50,50,50,50,50,50,50
hispanic,903,903,903,903,903,903,903,903,903,903,903,903,903,106,106,797,903,903,903,903,901,724,903,903,903,903,903,903,903,903
white,1582,1582,1582,1582,1582,1582,1581,1582,1582,1581,1581,1582,1582,212,212,1368,1582,1582,1582,1582,1578,1262,1582,1582,1582,1582,1582,1582,1582,1582
